# Python Task 2
## Question 1: Distance Matrix Calculation

In [16]:
import pandas as pd
import numpy as np

def calculate_distance_matrix(df):
    """
    Calculate a distance matrix based on the dataframe, df.

    Args:
        df (pandas.DataFrame): Input DataFrame containing information about distances between toll locations.

    Returns:
        pandas.DataFrame: Distance matrix
    """
    # Create a pivot table to represent distances between toll locations
    distance_matrix = df.pivot_table(index='id_start', columns='id_end', values='distance', fill_value=0)

    # Convert the pivot table to a DataFrame
    distance_matrix = pd.DataFrame(distance_matrix.to_numpy(), index=distance_matrix.index, columns=distance_matrix.columns)

    # Make the distance matrix symmetric by adding transpose values
    distance_matrix = distance_matrix.add(distance_matrix.T, fill_value=0)

    return distance_matrix

# Adjust the column names based on your specific dataset structure
df = pd.read_csv(r"C:\Users\amrit\Downloads\MapUp-Data-Assessment-F-main\MapUp-Data-Assessment-F-main\datasets\dataset-3.csv")
result_distance_matrix = calculate_distance_matrix(df)

# Print the result distance matrix
print(result_distance_matrix)

         1001400  1001402  1001404  1001406  1001408  1001410  1001412  \
1001400      NaN      9.7      0.0      0.0      0.0      0.0      0.0   
1001402      9.7      0.0     20.2      0.0      0.0      0.0      0.0   
1001404      0.0     20.2      0.0     16.0      0.0      0.0      0.0   
1001406      0.0      0.0     16.0      0.0     21.7      0.0      0.0   
1001408      0.0      0.0      0.0     21.7      0.0     11.1      0.0   
1001410      0.0      0.0      0.0      0.0     11.1      0.0     15.6   
1001412      0.0      0.0      0.0      0.0      0.0     15.6      0.0   
1001414      0.0      0.0      0.0      0.0      0.0      0.0     18.2   
1001416      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001418      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001420      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001422      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1001424      0.0      0.0      0.0    

# Question 2: Unroll Distance Matrix

In [34]:
import pandas as pd

def unroll_distance_matrix(distance_matrix):
    # Reset the index to include id_start as a column
    distance_matrix_reset = distance_matrix.reset_index()

    # Melt the DataFrame to create the desired format
    unrolled_df = pd.melt(distance_matrix_reset, id_vars=['index'], var_name='id_end', value_name='distance')

    # Rename columns for consistency
    unrolled_df.columns = ['id_start', 'id_end', 'distance']

    # Remove rows where id_start is equal to id_end
    unrolled_df = unrolled_df[unrolled_df['id_start'] != unrolled_df['id_end']]

    # Reset the index of the resulting DataFrame
    unrolled_df.reset_index(drop=True, inplace=True)

    return unrolled_df

# Example usage with the distance matrix obtained from the previous question
resulting_matrix = calculate_distance_matrix(csv_file_path)
unrolled_result = unroll_distance_matrix(resulting_matrix)
print(unrolled_result)

      id_start   id_end distance
0      1001402  1001400      9.7
1      1001404  1001400     29.9
2      1001406  1001400     45.9
3      1001408  1001400     67.6
4      1001410  1001400     78.7
...        ...      ...      ...
1801   1001470  1004356    159.8
1802   1001472  1004356    175.8
1803   1001488  1004356      4.0
1804   1004354  1004356      2.0
1805   1004355  1004356      4.0

[1806 rows x 3 columns]


# Question 3: Finding IDs within Percentage Threshold

In [68]:
import pandas as pd

def find_ids_within_ten_percentage_threshold(distance_matrix, reference_value):
    # Filter the DataFrame based on the reference value in id_start
    subset_df = distance_matrix[distance_matrix['id_start'] == reference_value]

    # Check if the reference value exists in the DataFrame
    if subset_df.empty:
        print(f"No data found for id_start = {reference_value}")
        return []

    # Calculate the average distance for the reference value
    average_distance = subset_df['distance'].mean()

    # Calculate the threshold range (10% of the average distance)
    threshold = 0.1 * average_distance

    # Find ids within the threshold (including ceiling and floor)
    within_threshold_ids = distance_matrix[
        (distance_matrix['distance'] >= average_distance - threshold) & 
        (distance_matrix['distance'] <= average_distance + threshold)
    ]['id_start'].unique()

    # Sort the list of ids
    sorted_within_threshold_ids = sorted(within_threshold_ids)

    return sorted_within_threshold_ids

# Example usage with the distance matrix obtained from the previous question
resulting_matrix = calculate_distance_matrix(csv_file_path)
unrolled_result = unroll_distance_matrix(resulting_matrix)

# Example with a reference value (replace 1 with your desired reference value)
reference_value = 1001406
within_threshold_ids = find_ids_within_ten_percentage_threshold(unrolled_result, reference_value)
print(within_threshold_ids)

[1001400, 1001402, 1001404, 1001406, 1001408, 1001410, 1001412, 1001414, 1001416, 1001418, 1001420, 1001422, 1001424, 1001426, 1001428, 1001430, 1001432, 1001434, 1001436, 1001437, 1001438, 1001440, 1001442, 1001444, 1001446, 1001448, 1001450, 1001452, 1001454, 1001456, 1001458, 1001460, 1001461, 1001462, 1001464, 1001466, 1001468, 1001470, 1001472, 1001488, 1004354, 1004355, 1004356]


# Question 4: Calculate Toll Rate


In [69]:
import pandas as pd

def calculate_toll_rate(distance_matrix):
    # Copy the input DataFrame to avoid modifying the original
    result_with_toll = distance_matrix.copy()

    # Define rate coefficients for each vehicle type
    rate_coefficients = {'moto': 0.8, 'car': 1.2, 'rv': 1.5, 'bus': 2.2, 'truck': 3.6}

    # Calculate toll rates for each vehicle type
    for vehicle_type, rate_coefficient in rate_coefficients.items():
        result_with_toll[vehicle_type] = result_with_toll['distance'] * rate_coefficient

    return result_with_toll

# Example usage with the unrolled DataFrame obtained from the previous question
csv_file_path = r"C:\Users\amrit\Downloads\MapUp-Data-Assessment-F-main\MapUp-Data-Assessment-F-main\datasets\dataset-3.csv"
resulting_matrix = calculate_distance_matrix(csv_file_path)
unrolled_result = unroll_distance_matrix(resulting_matrix)

# Calculate toll rates for each vehicle type
result_with_toll = calculate_toll_rate(unrolled_result)
print(result_with_toll)

      id_start   id_end distance    moto     car      rv     bus   truck
0      1001402  1001400      9.7    7.76   11.64   14.55   21.34   34.92
1      1001404  1001400     29.9   23.92   35.88   44.85   65.78  107.64
2      1001406  1001400     45.9   36.72   55.08   68.85  100.98  165.24
3      1001408  1001400     67.6   54.08   81.12   101.4  148.72  243.36
4      1001410  1001400     78.7   62.96   94.44  118.05  173.14  283.32
...        ...      ...      ...     ...     ...     ...     ...     ...
1801   1001470  1004356    159.8  127.84  191.76   239.7  351.56  575.28
1802   1001472  1004356    175.8  140.64  210.96   263.7  386.76  632.88
1803   1001488  1004356      4.0     3.2     4.8     6.0     8.8    14.4
1804   1004354  1004356      2.0     1.6     2.4     3.0     4.4     7.2
1805   1004355  1004356      4.0     3.2     4.8     6.0     8.8    14.4

[1806 rows x 8 columns]


# Question 5: Calculate Time-Based Toll Rates

In [123]:
import pandas as pd
from datetime import time, timedelta

def calculate_time_based_toll_rates(df):
    # Define time ranges and discount factors
    time_ranges_weekdays = [(time(0, 0, 0), time(10, 0, 0)), (time(10, 0, 0), time(18, 0, 0)), (time(18, 0, 0), time(23, 59, 59))]
    time_ranges_weekends = [(time(0, 0, 0), time(23, 59, 59))]
    discount_factors_weekdays = [0.8, 1.2, 0.8]
    discount_factor_weekends = 0.7

    # Initialize empty DataFrame to store the results
    result_with_time_based_tolls = pd.DataFrame()

    # Iterate over unique (id_start, id_end) pairs
    unique_pairs = df[['id_start', 'id_end']].drop_duplicates()
    for index, row in unique_pairs.iterrows():
        id_start = row['id_start']
        id_end = row['id_end']

        # Create time-based toll rates for weekdays
        for start_time, end_time, discount_factor in zip(time_ranges_weekdays, time_ranges_weekdays[1:], discount_factors_weekdays):
            result_subset = df[(df['id_start'] == id_start) & (df['id_end'] == id_end)].copy()
            result_subset['start_day'] = 'Monday'
            result_subset['end_day'] = 'Sunday'
            result_subset['start_time'] = start_time[0]
            result_subset['end_time'] = end_time[1]
            result_subset['toll_rate'] = result_subset['distance'] * discount_factor
            result_with_time_based_tolls = pd.concat([result_with_time_based_tolls, result_subset], ignore_index=True)

        # Create time-based toll rates for weekends
        for start_time, end_time in time_ranges_weekends:
            result_subset = df[(df['id_start'] == id_start) & (df['id_end'] == id_end)].copy()
            result_subset['start_day'] = 'Monday'
            result_subset['end_day'] = 'Sunday'
            result_subset['start_time'] = start_time
            result_subset['end_time'] = end_time
            result_subset['toll_rate'] = result_subset['distance'] * discount_factor_weekends
            result_with_time_based_tolls = pd.concat([result_with_time_based_tolls, result_subset], ignore_index=True)

    return result_with_time_based_tolls

# Example usage with the DataFrame containing toll rates obtained from the previous question
result_with_toll_rates = calculate_toll_rate(unrolled_result)

# Calculate time-based toll rates
result_with_time_based_tolls = calculate_time_based_toll_rates(result_with_toll_rates)
print(result_with_time_based_tolls)

      id_start   id_end distance   moto    car     rv    bus   truck  \
0      1001402  1001400      9.7   7.76  11.64  14.55  21.34   34.92   
1      1001402  1001400      9.7   7.76  11.64  14.55  21.34   34.92   
2      1001402  1001400      9.7   7.76  11.64  14.55  21.34   34.92   
3      1001404  1001400     29.9  23.92  35.88  44.85  65.78  107.64   
4      1001404  1001400     29.9  23.92  35.88  44.85  65.78  107.64   
...        ...      ...      ...    ...    ...    ...    ...     ...   
5413   1004354  1004356      2.0    1.6    2.4    3.0    4.4     7.2   
5414   1004354  1004356      2.0    1.6    2.4    3.0    4.4     7.2   
5415   1004355  1004356      4.0    3.2    4.8    6.0    8.8    14.4   
5416   1004355  1004356      4.0    3.2    4.8    6.0    8.8    14.4   
5417   1004355  1004356      4.0    3.2    4.8    6.0    8.8    14.4   

     start_day end_day start_time  end_time toll_rate  
0       Monday  Sunday   00:00:00  18:00:00      7.76  
1       Monday  Sunday 